# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.u

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355117736503                   -1.09    6.2    165ms
  2   -9.356772439786       -2.78       -1.44    1.0   89.0ms
  3   -9.357067062840       -3.53       -2.75    2.4   96.7ms
  4   -9.357119538866       -4.28       -3.00    6.5    524ms
  5   -9.357119894826       -6.45       -3.14    2.5    100ms
  6   -9.357120107290       -6.67       -3.29    1.0   79.6ms
  7   -9.357120235486       -6.89       -3.46    1.0   81.1ms
  8   -9.357120287904       -7.28       -3.62    1.0   82.4ms
  9   -9.357120315501       -7.56       -3.89    1.1   81.7ms
 10   -9.357120321804       -8.20       -4.14    2.5   95.4ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.74266337673                   -0.98    6.4    503ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     1
│     1
│     1
│     1
│     1
│     6
│     1
│     1
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  2   -18.74067798547   +   -2.70       -1.37    1.6    273ms
  3   -18.79226543486       -1.29       -1.74    5.4    429ms
  4   -18.79231568894       -4.30       -1.80    2.6    304ms
  5   -18.79124435183   +   -2.97       -2.11    2.2    276ms
  6   -18.79239460605       -2.94       -2.48    1.2    281ms
  7   -18.79256202007       -3.78       -2.72    3.1    309ms
  8   -18.79254534684   +   -4.78       -2.71    1.9    242ms
  9   -18.79257152063       -4.58       -2.80    1.0    217ms
 10   -18.79260352720       -4.49       -2.98    1.1    243ms
 11   -18.79261959256       -4.79       -3.27

In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     12
│      3
│     16
│      3
│     16
│     11
│     14
│      2
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.53140486655                   -0.85    9.6    2.10s
  2   -37.55602340450       -1.61       -1.18    2.8    862ms
  3   -37.56419855628       -2.09       -2.13   13.2    1.74s
  4   -37.56493836588       -3.13       -2.40    9.0    1.65s
  5   -37.56498005173       -4.38       -3.25    1.8    932ms
  6   -37.56498461415       -5.34       -3.65    9.8    2.02s
  7   -37.56498516470       -6.26       -4.24    3.1    1.00s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.350339045050                   -1.10    6.0    180ms
  2   -9.356798465204       -2.19       -1.82    1.0   68.4ms
  3   -9.357075783349       -3.56       -2.49    5.5    156ms
  4   -9.356983521159   +   -4.03       -2.19    3.4    128ms
  5   -9.357119517691       -3.87       -3.65    1.1   91.5ms
  6   -9.357119949576       -6.36       -3.54    5.2    159ms
  7   -9.357120300628       -6.45       -4.09    1.1   73.4ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54143125874                   -0.84    9.2    1.88s
  2   -37.55074022589       -2.03       -1.49    3.0    735ms
  3   -35.81943598901   +    0.24       -0.87   11.4    1.69s
  4   -28.81342633284   +    0.85       -0.60    6.8    1.84s
  5   -37.29785421109        0.93       -1.26    6.4    1.36s
  6   -37.52507982064       -0.64       -1.73    3.8    999ms
  7   -36.46641287678   +    0.02       -1.05    4.8    1.16s
  8   -37.56151851901        0.04       -2.19    6.1    1.52s
  9   -37.56462281644       -2.51       -2.32    3.0    1.06s
 10   -37.56480966520       -3.73       -2.63    1.6    825ms
 11   -37.56466704757   +   -3.85       -2.53    2.6    1.04s
 12   -37.56491333555       -3.61       -2.91    1.8    871ms
 13   -37.56492803260       -4.83       -3.05    2.0    1.06s
 14   -37.56496662922       -4.41       -3.28    1.8    764ms
 15   -37

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.